In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
from tabulate import tabulate
from scipy import stats

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Year 3 Sems 3/Data Analysis Project/Case Study/cleaned data.csv")

In [ ]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2.0,h,707435.52,SS,Jellis,3/9/2016,2.5,3067.0,...,1.0,1.0,126.0,84.3,1900.0,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2.0,h,1480000.00,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,194.2,1900.0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2.0,h,1035000.00,S,Biggin,4/2/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3.0,u,1186997.35,VB,Rounds,4/2/2016,2.5,3067.0,...,2.0,1.0,0.0,80.0,1900.0,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3.0,h,1465000.00,SP,Biggin,4/3/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
#add in column for log price
data["Log_Price"] = np.log(data["Price"])

# Convert the Date column to a datetime data type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

#add in column for year sold
data["YearSold"] = data["Date"].dt.year

#add in column for the month sold
data["MonthSold"] = data["Date"].dt.month

#compute the age of house
age = list()
for counter,year in enumerate(data["YearBuilt"]):
  #we take 2018 is because the dataset is collected at year 2018
  #and for any age that is negative, we straight away convert it to become 0
  tempAge = 2018 - year
  if tempAge < 0:
    age.append(0)
  else:
    age.append(tempAge)

#inserting house age into new column of dataframe
if "Age" not in data.columns:
  data.insert(len(data.columns), "Age", age)

#modify the space and dash in regionname column to be replaced with underscore
data["Regionname"] = data["Regionname"].str.replace(" ","_")
data["Regionname"] = data["Regionname"].str.replace("-","_")

In [ ]:
#perform one hot encoding onto type column
#to break categorical variable into a few columns
#This is for ease of comparison later on
data = pd.get_dummies(data,columns=["Type","Regionname"])

In [ ]:
data.head()

,Suburb,Address,Rooms,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,...,Type_t,Type_u,Regionname_Eastern_Metropolitan,Regionname_Eastern_Victoria,Regionname_Northern_Metropolitan,Regionname_Northern_Victoria,Regionname_South_Eastern_Metropolitan,Regionname_Southern_Metropolitan,Regionname_Western_Metropolitan,Regionname_Western_Victoria
0,Abbotsford,68 Studley St,2.0,707435.52,SS,Jellis,2016-09-03,2.5,3067.0,2.0,...,0,0,0,0,1,0,0,0,0,0
1,Abbotsford,85 Turner St,2.0,1480000.00,S,Biggin,2016-12-03,2.5,3067.0,2.0,...,0,0,0,0,1,0,0,0,0,0
2,Abbotsford,25 Bloomburg St,2.0,1035000.00,S,Biggin,2016-02-04,2.5,3067.0,2.0,...,0,0,0,0,1,0,0,0,0,0
3,Abbotsford,18/659 Victoria St,3.0,1186997.35,VB,Rounds,2016-02-04,2.5,3067.0,3.0,...,0,1,0,0,1,0,0,0,0,0
4,Abbotsford,5 Charles St,3.0,1465000.00,SP,Biggin,2017-03-04,2.5,3067.0,3.0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
#list of independent variable
xV = ["Rooms","Distance","Bedroom2","Bathroom","Landsize","BuildingArea","MonthSold","YearSold","Age","Type_h","Type_t","Type_u","Propertycount","Regionname_Eastern_Metropolitan","Regionname_Eastern_Victoria","Regionname_Northern_Metropolitan","Regionname_Northern_Victoria","Regionname_South_Eastern_Metropolitan","Regionname_Southern_Metropolitan","Regionname_Western_Metropolitan","Regionname_Western_Victoria"]

#list of dependent variable
yV = ["Log_Price"]

#list of independent and dependent variable
allV = xV + yV

#linear equation
formula = 'Log_Price ~ Rooms + Bedroom2 + Distance + Bathroom + Landsize + BuildingArea + MonthSold + YearSold + Age + Type_h + Type_t + Type_u + Propertycount + Regionname_Eastern_Metropolitan + Regionname_Eastern_Victoria + Regionname_Northern_Metropolitan + Regionname_Northern_Victoria + Regionname_South_Eastern_Metropolitan + Regionname_Southern_Metropolitan + Regionname_Western_Metropolitan + Regionname_Western_Victoria'

print(allV)

['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Landsize', 'BuildingArea', 'MonthSold', 'YearSold', 'Age', 'Type_h', 'Type_t', 'Type_u', 'Propertycount', 'Regionname_Eastern_Metropolitan', 'Regionname_Eastern_Victoria', 'Regionname_Northern_Metropolitan', 'Regionname_Northern_Victoria', 'Regionname_South_Eastern_Metropolitan', 'Regionname_Southern_Metropolitan', 'Regionname_Western_Metropolitan', 'Regionname_Western_Victoria', 'Log_Price']


In [ ]:
#visualize the correlation matrix
data[allV].corr()

,Rooms,Distance,Bedroom2,Bathroom,Landsize,BuildingArea,MonthSold,YearSold,Age,Type_h,...,Propertycount,Regionname_Eastern_Metropolitan,Regionname_Eastern_Victoria,Regionname_Northern_Metropolitan,Regionname_Northern_Victoria,Regionname_South_Eastern_Metropolitan,Regionname_Southern_Metropolitan,Regionname_Western_Metropolitan,Regionname_Western_Victoria,Log_Price
Rooms,1.000000,0.183987,0.959815,0.385971,0.021793,0.075357,0.012414,0.083815,-0.024923,0.306185,...,-0.077061,0.113425,0.018325,-0.119546,0.025546,0.037791,-0.009022,0.019226,0.013122,0.366856
Distance,0.183987,1.000000,0.186249,0.094051,0.032781,0.055844,0.012044,0.231275,-0.238787,0.216399,...,-0.018135,0.192591,0.272960,-0.186817,0.252682,0.439855,-0.253161,-0.018105,0.167196,-0.095942
Bedroom2,0.959815,0.186249,1.000000,0.389636,0.021919,0.073110,0.013975,0.097915,-0.028986,0.294561,...,-0.076733,0.114025,0.019433,-0.117164,0.026124,0.040110,-0.010974,0.016690,0.013649,0.353572
Bathroom,0.385971,0.094051,0.389636,1.000000,0.032074,0.094365,0.010213,0.044609,-0.080499,0.146451,...,-0.052004,0.082014,0.027989,-0.148248,0.016949,0.011249,0.099348,-0.035438,-0.000721,0.540207
Landsize,0.021793,0.032781,0.021919,0.032074,1.000000,0.417273,0.001197,0.022567,-0.020496,0.017529,...,-0.004854,0.009869,0.017755,-0.007811,0.024508,0.016057,-0.007418,-0.008008,0.001892,0.028033
BuildingArea,0.075357,0.055844,0.073110,0.094365,0.417273,1.000000,-0.001426,0.026723,-0.018528,0.053724,...,-0.006733,0.016759,0.024696,-0.026685,0.012607,0.030639,-0.002568,-0.004825,-0.002477,0.082866
MonthSold,0.012414,0.012044,0.013975,0.010213,0.001197,-0.001426,1.000000,-0.490577,0.046250,0.016517,...,0.003921,0.011507,-0.000160,-0.007310,-0.003446,0.019805,-0.000942,-0.011020,0.003904,0.011852
YearSold,0.083815,0.231275,0.097915,0.044609,0.022567,0.026723,-0.490577,1.000000,-0.056231,0.059619,...,0.011816,0.084889,0.048951,-0.008652,0.054821,0.115047,-0.126900,0.002002,0.032567,0.001866
Age,-0.024923,-0.238787,-0.028986,-0.080499,-0.020496,-0.018528,0.046250,-0.056231,1.000000,0.123785,...,-0.021001,-0.072256,-0.029254,0.023820,-0.046281,-0.040342,0.111133,-0.059148,-0.021166,0.153530
Type_h,0.306185,0.216399,0.294561,0.146451,0.017529,0.053724,0.016517,0.059619,0.123785,1.000000,...,-0.060054,0.079890,0.045433,-0.011923,0.048294,0.045402,-0.153621,0.081459,0.036590,0.271528


By referring to the result of correlation matrix table above, it shows that the relationship between Rooms and Bedroom2 is quite high, therefore we will choose either one variable from it. And we will choose Rooms variable due to having higher relationship with Log_Price variable which is our dependent variable

**Lee: start from here.**

In [ ]:
#remove bedroom2 variable
if "Bedroom2" in xV:
  xV.remove("Bedroom2")
  allV = xV + yV
  # Remove "+ Bedroom2" from the formula
  formula = formula.replace(" + Bedroom2", "")

In [ ]:
# Standardize beta coefficient method

# standardizing dataframe
  # select all columns in dataframe which contains only numeric value
  # drop the null value
  # and compute the relative z-score
df_z = data.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

# fitting regression
result = smf.ols(formula, data=df_z).fit()

# checking results
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              Log_Price   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     1670.
Date:                Wed, 13 Sep 2023   Prob (F-statistic):               0.00
Time:                        07:07:21   Log-Likelihood:                -38612.
No. Observations:               34852   AIC:                         7.726e+04
Df Residuals:                   34833   BIC:                         7.742e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

From the result above, we notice that p-values for landsize, propertycount and monthSold variable is not significant, therefore, we decide to drop it and re-run the analysis again.**(Lee 4.4)**

In [ ]:
if "Landsize" in xV:
  xV.remove("Landsize")
  allV = xV + yV
  # Remove "+ Landsize" from the formula
  formula = formula.replace(" + Landsize", "")

if "Propertycount" in xV:
  xV.remove("Propertycount")
  allV = xV + yV
  # Remove "+ Propertycount" from the formula
  formula = formula.replace(" + Propertycount", "")

if "MonthSold" in xV:
  xV.remove("MonthSold")
  allV = xV + yV
  # Remove "+ MonthSold" from the formula
  formula = formula.replace(" + MonthSold", "")

In [ ]:
# Standardize beta coefficient method

# standardizing dataframe
  # select all columns in dataframe which contains only numeric value
  # drop the null value
  # and compute the relative z-score
df_z = data.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

# fitting regression
result = smf.ols(formula, data=df_z).fit()

# checking results
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              Log_Price   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     2003.
Date:                Wed, 13 Sep 2023   Prob (F-statistic):               0.00
Time:                        07:07:21   Log-Likelihood:                -38614.
No. Observations:               34852   AIC:                         7.726e+04
Df Residuals:                   34836   BIC:                         7.740e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

From the result above, it provides lots of valuable information for us to make informed decisions. R square of 0.463 indicates that 46.30% of variability in dependent variable can be explained by the independent variable. This shows a moderate relationship between independent variable and dependent variable. Since all the p-values of independent variable is less than 0.05, we can safely assume that all variable is significant. Apart from that, we can also come out with a linear equation to predict the y variable value based on multiple x variables.

Equation: Log_Price=(−7.18×10
−16
 )+(0.144⋅Rooms)+(−0.227⋅Distance)+(0.431⋅Bathroom)+(0.026⋅BuildingArea)+(0.108⋅Age)+(0.116⋅Type_h)+(0.013⋅Type_t)+(−0.141⋅Type_u)+(0.043⋅Regionname_Eastern_Metropolitan)+(0.039⋅Regionname_Eastern_Victoria)+(−0.098⋅Regionname_Northern_Metropolitan)+(0.017⋅Regionname_Northern_Victoria)+(0.052⋅Regionname_South_Eastern_Metropolitan)+(0.120⋅Regionname_Southern_Metropolitan)+(−0.105⋅Regionname_Western_Metropolitan)+(−0.026⋅Regionname_Western_Victoria)

**(Lee : 4.4)**

In [ ]:
#gather required information
mergedLst = list()
for i in range(len(xV)):
  lst = list((xV[i],round(result.pvalues[i+1],3),round(result.params[i+1],3),abs(round(result.params[i+1],3))))
  mergedLst.append(list(lst))

In [ ]:
#rank independent variable according to absolute standardize beta coefficient
sortedLst = sorted(mergedLst,key=lambda x:x[3],reverse=True)

In [ ]:
#tabulate the finding
col_names = ["Content","p-values","Coefficient","Absolute Coefficient"]
print(tabulate(sortedLst, headers=col_names, tablefmt="fancy_grid", showindex="always"))

╒════╤═══════════════════════════════════════╤════════════╤═══════════════╤════════════════════════╕
│    │ Content                               │   p-values │   Coefficient │   Absolute Coefficient │
╞════╪═══════════════════════════════════════╪════════════╪═══════════════╪════════════════════════╡
│  0 │ Bathroom                              │          0 │         0.431 │                  0.431 │
├────┼───────────────────────────────────────┼────────────┼───────────────┼────────────────────────┤
│  1 │ Distance                              │          0 │        -0.231 │                  0.231 │
├────┼───────────────────────────────────────┼────────────┼───────────────┼────────────────────────┤
│  2 │ Rooms                                 │          0 │         0.143 │                  0.143 │
├────┼───────────────────────────────────────┼────────────┼───────────────┼────────────────────────┤
│  3 │ Type_u                                │          0 │        -0.141 │                

The table shows the ranking of variable according to the standardized beta coefficient of independent variable(absolute coefficient). From this table, we get to knows that Bathroom variable is having the most significant effect onto the changes of dependent variable which is Price variable. Since it is having positive relationship, it means that as number of bathroom increase, the log price variable increase as well. Apart from that, distance variable also having great influence onto the log price variables. It shows us that as one house is getting close to the city, the log price is higher. For comparison among house type, it shows that type_u is having a higher coefficient value compare to other type. By analyzing the relationship, it tells us that unit house type is more cheaper than other house type. On the hand, if we compare among the region, it can clearly be seen that coefficient value of southern metropolitan is ahead of other region and since the relationship is positive, it means that the housing price at this area is more expensive than other area.**(Lee: Chapter 4.5)**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
#perform machine learning to verify our computed r-squared
x_train, x_test, y_train, y_test = train_test_split(data[xV],data[yV], test_size = 0.20, random_state= 42)

In [ ]:
# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(x_train, y_train)

LinearRegression()

In [ ]:
# Make predictions on the test data
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score

# Calculate R2
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

R2 Score: 0.4494157855919174


Since the r-squared that we computed from the OLS method which is 0.463 is close to the r-squared computed by the machine learning model, which is 0.449. We can assume that this model is well fit and therefore, we can verified that the computed linear regression is accurate.